In [2]:
# We're doing some basic deduplication to remove exact matches for street number 
# + strong fuzzy matches for street names
# within the same city


# NOTE if I'm deduplicating, I need to be careful to retain data
# specifically for now, I want to keep the one with a geocode

# I'll need to sort the data first perhaps - https://stackoverflow.com/questions/43814487/remove-duplicate-rows-in-pandas-dataframe-based-on-condition
# then the nan ones for geo will be lower

In [1]:
import pandas as pd

from thefuzz import fuzz
from thefuzz import process

/home/jovyan/.local/lib/python3.8/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [10]:
df = pd.read_csv('data/formatted.csv', low_memory=False)
print('rows: ', len(df))

rows:  803584


In [11]:
print(len(df[df['longitude'].isnull()]))

598853


In [13]:
df['latitude'].tail(30)

803554   NaN
803555   NaN
803556   NaN
803557   NaN
803558   NaN
803559   NaN
803560   NaN
803561   NaN
803562   NaN
803563   NaN
803564   NaN
803565   NaN
803566   NaN
803567   NaN
803568   NaN
803569   NaN
803570   NaN
803571   NaN
803572   NaN
803573   NaN
803574   NaN
803575   NaN
803576   NaN
803577   NaN
803578   NaN
803579   NaN
803580   NaN
803581   NaN
803582   NaN
803583   NaN
Name: latitude, dtype: float64

In [17]:
# Start by removing exact matches

# df = df.drop_duplicates(subset=['street_no','formatted_en', 'province'])
# df = df[(~df.duplicated(subset=['street_no','formatted_en', 'province'])) | df['street_no','formatted_en', 'province'].isna()]

# sort so that we keep records of the lat/lon if present
df.sort_values(['latitude'])
# this only seems to change the result by 20. Does that make sense? That suggests that pretty much all of the duplicates have the same lat/lon information

df_city = df[~df.duplicated(subset=['street_no','formatted_en', 'province', 'city'], keep='first') | df['street_no'].isna() | df['formatted_en'].isna() | df['province'].isna() | df['city'].isna()]
# df_province = df[~df.duplicated(subset=['street_no','formatted_en', 'province']) | df['street_no'].isna() | df['formatted_en'].isna() | df['province'].isna()]
# df_nogeo = df[~df.duplicated(subset=['street_no','formatted_en']) | df['street_no'].isna() | df['formatted_en'].isna()]

# FOR TESTING create a dataframe of duplicates for inspection

print('rows: ', len(df_city))
# print('rows: ', len(df_province))
# print('rows: ', len(df_nogeo))
# if (province_code == 'QC'):
#     df = df.drop_duplicates(subset=['street_no','formatted_fr'])
# else:
#     df = df.drop_duplicates(subset=['street_no','formatted_en'])

rows:  484011


In [20]:
duplicates = df[df.duplicated(subset=['street_no','formatted_en', 'province', 'city'], keep=False)]

In [31]:
duplicates = duplicates.sort_values(['street_no','formatted_en', 'province', 'city'])
duplicates[['street_no','formatted_en', 'province', 'city']].head(30)

,street_no,formatted_en,province,city
578989,#,bellamy rd n,ON,"TORONTO, ON"
570656,#,bellamy rd n,ON,"TORONTO, ON"
577458,#,bellamy rd n,ON,"TORONTO, ON"
575168,#,bellamy rd n,ON,"TORONTO, ON"
575276,#,bellamy rd n,ON,"TORONTO, ON"
563142,#,bellamy rd n,ON,"TORONTO, ON"
573346,#,bellamy rd n,ON,"TORONTO, ON"
592282,#,bellamy rd n,ON,"TORONTO, ON"
560760,#,bellamy rd n,ON,"TORONTO, ON"
581196,#,bellamy rd n,ON,"TORONTO, ON"


In [18]:
# how many to geocode?
print(len(df_city[df_city['latitude'].isnull()]))

398644


In [8]:
df_city['province'].unique()

array(['BC', 'AB', 'NB', 'ON', nan, 'QC', 'MB', 'SK', 'NS', '78', 'YT',
       'NL', 'LAKEVIEW ESTATES', 'NT', 'NU', 'PE'], dtype=object)

In [12]:
df_city[(df_city['province'] == 'LAKEVIEW ESTATES') | (df_city['province'] == '78')]

,idx,localfile,business_name,business_sector,business_subsector,business_description,business_id_no,licence_number,licence_type,primary_NAICS,...,LP_Unit,LP_street_no_alt,LP2_unit,LP2_street_no,spill,LP3_unit,parsing_err_exists,flagged_parsing_err,formatted_en,formatted_fr
101179,7a122c15a250341d251a,BC_Vancouver_Business_Licences.csv,Post The Butcher Inc,Office,Broker,NaN,NaN,19-138327,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN,NaN
184292,69139e8fad512fe66792,BC_Vancouver_Business_Licences.csv,(Naoko Winther),Health Services,Counselling,NaN,NaN,22-123793,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN,NaN


In [36]:
len(df)

803584

In [35]:
df['full_address'].isna().sum()

0

In [34]:
list(df)

['idx',
 'localfile',
 'business_name',
 'business_sector',
 'business_subsector',
 'business_description',
 'business_id_no',
 'licence_number',
 'licence_type',
 'primary_NAICS',
 'secondary_NAICS',
 'NAICS_descr',
 'alt_econ_act_code',
 'alt_econ_act_descrip',
 'latitude',
 'longitude',
 'full_address',
 'full_address_2',
 'mailing_address',
 'postal_code',
 'unit',
 'street_no',
 'street_name',
 'street_direction',
 'street_type',
 'city',
 'province',
 'country',
 'business_website',
 'email',
 'telephone',
 'telephone_extension',
 'toll_free_telephone',
 'fax',
 'total_no_employees',
 'no_full_time',
 'no_part_time',
 'no_seasonal',
 'date_established',
 'indigenous',
 'status',
 'provider',
 'geo_source',
 'LP_street_no',
 'LP_street_name',
 'LP_City',
 'LP_Province',
 'LP_PostCode',
 'LP_Unit',
 'LP_street_no_alt',
 'LP2_unit',
 'LP2_street_no',
 'spill',
 'LP3_unit',
 'parsing_err_exists',
 'flagged_parsing_err',
 'formatted_en',
 'formatted_fr']